# MaiChart AI - Colab Training (Deep Optimized)

1. **Check GPU**: Runtime -> Change runtime type -> T4 GPU.
2. **Features**:
    *   **Auto-Resume**: 自动寻找最新的 `epoch_X.pt` 并接力轮数训练。
    *   **Cloud Priority**: 挂载网盘后自动保存到 `MyDrive/maimai/models`。
    *   **Auto-Download**: 即使不挂载网盘，每轮训练完也会自动触发浏览器下载保存权重。
    *   **Full States**: `best.pt` 也包含优化器状态，支持完美恢复。


In [1]:
!nvidia-smi

Wed Jan 21 18:03:59 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os

# 1. 尝试挂载 Google Drive (如果环境支持)
try:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        print("正在尝试挂载 Google Drive...")
        drive.mount('/content/drive')
    HAS_DRIVE = os.path.exists('/content/drive')
except Exception as e:
    print(f"⚠️ 无法挂载 Google Drive: {e}")
    print("将使用本地 /content 路径进行训练。如果你有数据集，请手动上传到 /content 目录。")
    HAS_DRIVE = False

# 2. 确保回到根目录，彻底清理可能有问题的旧文件夹
%cd /content
!rm -rf /content/maimai-chat-ai

# 3. 重新克隆仓库
!git clone https://github.com/lvyirgil/maimai-chat-ai.git

# 4. 进入文件夹
%cd /content/maimai-chat-ai

# 5. 检查文件列表
print("\n--- 当前目录下的文件 ---")
!ls

# 6. 安装依赖
if os.path.exists('requirements.txt'):
    !pip install -r requirements.txt
else:
    print("\n❌ 错误：仓库里依然没有找到 requirements.txt")

/content
Cloning into 'maimai-chat-ai'...
remote: Enumerating objects: 1883, done.
remote: Counting objects: 100% (1883/1883), done.
remote: Compressing objects: 100% (1840/1840), done.
remote: Total 1883 (delta 42), reused 1874 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (1883/1883), 4.39 MiB | 17.21 MiB/s, done.
Resolving deltas: 100% (42/42), done.
/content/maimai-chat-ai

--- 当前目录下的文件 ---
check_cuda.py  GPU_TRAINING.md		 README.md	   tests
configs        IMPORT_TOOL_INSTALLED.md  requirements.txt  train_gpu.bat
data	       models			 scripts	   train_gpu.py
docs	       notebooks		 src		   train.ps1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 86.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [ ]:
import zipfile
import os
import shutil

# 定义可能的几个路径
zip_candidates = [
    '/content/drive/MyDrive/maimai_data.zip',
    '/content/maimai_data.zip',
    '/content/drive/MyDrive/maimai/maimai_data.zip'
]

target_base = '/content/maimai-chat-ai/data'

def force_unzip():
    found_zip = None
    for p in zip_candidates:
        if os.path.exists(p):
            found_zip = p
            break
    
    if not found_zip:
        print("❌ 错误：未找到权重压缩包 maimai_data.zip")
        print("请确保已进行以下操作之一：")
        print("1. 挂载了网盘且文件在 MyDrive 根目录")
        print("2. 手动将文件上传到了 Colab 左侧文件栏的 /content 目录下")
        return

    print(f"📦 发现压缩包: {found_zip}")

    # 先清理一遍
    shutil.rmtree(os.path.join(target_base, 'audio'), ignore_errors=True)
    shutil.rmtree(os.path.join(target_base, 'raw'), ignore_errors=True)
    os.makedirs(os.path.join(target_base, 'audio'), exist_ok=True)
    os.makedirs(os.path.join(target_base, 'raw'), exist_ok=True)

    print("正在强力解压中，请稍后...")
    with zipfile.ZipFile(found_zip, 'r') as z:
        for info in z.infolist():
            try:
                name = info.filename.encode('cp437').decode('gbk')
            except:
                name = info.filename
            name = name.replace('\\', '/')

            if name.endswith(('.mp3', '.wav', '.ogg', '.flac', '.MP3')):
                fname = os.path.basename(name)
                if fname:
                    with z.open(info) as source, open(os.path.join(target_base, 'audio', fname), 'wb') as target:
                        shutil.copyfileobj(source, target)
            elif name.endswith('.txt'):
                fname = os.path.basename(name)
                if fname:
                    with z.open(info) as source, open(os.path.join(target_base, 'raw', fname), 'wb') as target:
                        shutil.copyfileobj(source, target)

    print(f"✅ 解压完成！音频: {len(os.listdir(os.path.join(target_base, 'audio')))}, 谱面: {len(os.listdir(os.path.join(target_base, 'raw')))}")

force_unzip()

正在强力解压中，请稍后（4.6GB 可能需要 2-3 分钟）...
✅ 解压完成！
当前音频数: 1757
当前谱面数: 1810


In [9]:
# 1. 进入项目文件夹
%cd /content/maimai-chat-ai

# 2. 确认一下文件在不在（应该能看到 requirements.txt）
!ls

# 3. 再安装依赖
!pip install -r requirements.txt

/content/maimai-chat-ai
check_cuda.py  GPU_TRAINING.md		 README.md	   tests
configs        IMPORT_TOOL_INSTALLED.md  requirements.txt  train_gpu.bat
data	       models			 scripts	   train_gpu.py
docs	       notebooks		 src		   train.ps1


In [13]:
!python -m src.data.preprocess

警告: 找不到 M@GICAL☆CURE! LOVE ♥ SHOT! 的音频文件
警告: 找不到 Freak Out Hr 的音频文件
警告: 找不到 デビルじゃないもん 的音频文件
警告: 找不到 てらてら 的音频文件
警告: 找不到 Ref_rain (for 7th Heaven) 的音频文件
警告: 找不到 Colorfull_Encounter 的音频文件
警告: 找不到 美夜月鏡 的音频文件
警告: 找不到 MYTH Re：LEASE 的音频文件
警告: 找不到 はいよろこんで 的音频文件
警告: 找不到 sølips 的音频文件
警告: 找不到 廃墟にいますキャンペーン 的音频文件
警告: 找不到 Flashback 的音频文件
警告: 找不到 Hainuwele 的音频文件
警告: 找不到 抜錨 的音频文件
警告: 找不到 お呪い 的音频文件
警告: 找不到 [嘘] ライアーダンサー 的音频文件
警告: 找不到 Chronomia 的音频文件
警告: 找不到 あいたい星人 的音频文件
警告: 找不到 HYP3RTRIBE 的音频文件
警告: 找不到 ㋰責任集合体 的音频文件
警告: 找不到 カーニバルハッピー 的音频文件
警告: 找不到 唱 的音频文件
警告: 找不到 アンダーキッズ 的音频文件
警告: 找不到 ハオ 的音频文件
警告: 找不到 しゅ～しん？変身☆ハカイシンzzZ 的音频文件
警告: 找不到 Pixel Galaxy 的音频文件
警告: 找不到 偉大なる悪魔は実は大天使パトラちゃん様なのだ！ 的音频文件
警告: 找不到 マツケンサンバⅡ 的音频文件
警告: 找不到 Beginning together! 的音频文件
警告: 找不到 ビビデバ 的音频文件
警告: 找不到 [X] 人マニア 的音频文件
警告: 找不到 Lover_s Trick 的音频文件
警告: 找不到 つづみぐさ 的音频文件
警告: 找不到 サウンドプレイヤー 的音频文件
警告: 找不到 シスターシスター 的音频文件
警告: 找不到 FLΛME-FRΦST 的音频文件
警告: 找不到 Divide et impera! 的音频文件
警告: 找不到 シュガーホリック 的音频文件
警告: 找不到 NO ONE YES MAN 的音频文件
警告: 找不到 Bri

In [ ]:
import os
import glob

# 1. 配置路径
drive_models_path = '/content/drive/MyDrive/maimai/models'
local_models_path = '/content/maimai-chat-ai/models'

# 检查 Drive 是否真的可用
is_drive_ready = os.path.exists('/content/drive/MyDrive')

# 2. 同步逻辑
if is_drive_ready:
    print("📂 检测到 Google Drive，将开启自动同步...")
    !mkdir -p {drive_models_path}
    
    if os.path.exists(local_models_path) and not os.path.islink(local_models_path):
        print("正在将本地存量模型迁移至网盘...")
        !cp -rn {local_models_path}/* {drive_models_path}/ 2>/dev/null
        !rm -rf {local_models_path}
        !ln -s {drive_models_path} {local_models_path}
    elif not os.path.exists(local_models_path):
        !ln -s {drive_models_path} {local_models_path}
    print(f"✅ 同步已就绪: {drive_models_path}")
else:
    print("ℹ️ 未挂载网盘，模型将保存在本地: " + local_models_path)
    os.makedirs(local_models_path, exist_ok=True)

# 3. 寻找检查点
checkpoints = glob.glob(f"{local_models_path}/epoch_*.pt")
latest_resume = ""
if checkpoints:
    try:
        latest_resume = max(checkpoints, key=lambda x: int(os.path.basename(x).split('_')[1].split('.')[0]))
        print(f"🔄 检测到最新进度: {latest_resume}")
    except:
        pass
elif os.path.exists(f"{local_models_path}/best.pt"):
    latest_resume = f"{local_models_path}/best.pt"
    print("🔄 未找到 epoch 检查点，将从 best.pt 恢复。")

# 4. 启动训练
resume_cmd = f"--resume {latest_resume}" if latest_resume else ""
print(f"🚀 开始训练...")
!python train_gpu.py --config configs/default.yaml {resume_cmd} --epochs 100

正在建立实时同步软链接...
✅ 实时同步已就绪：现在模型保存会直接写入网盘。
载入配置文件: configs/default.yaml

GPU 信息
✓ CUDA 可用
✓ GPU 数量: 1

GPU 0: Tesla T4
  - 计算能力: 7.5
  - 总显存: 14.7 GB
  - 当前显存占用: 0.0 GB

✓ PyTorch 版本: 2.9.0+cu126
✓ 混合精度: 启用 (FP16)

训练配置:
  - 设备: cuda
  - Batch Size: 4
  - 学习率: 1e-4
  - 混合精度: 启用
  - 最大轮数: 100
  - 梯度累积步数: 4

✓ 检测到 GPU，启用混合精度和优化参数
加载样本: 100% 1757/1757 [00:52<00:00, 33.59it/s]
加载了 1757 个样本
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
从 /content/maimai-chat-ai/models/best.pt 加载检查点 (epoch 5)
开始训练，设备: cuda
训练集大小: 1405
验证集大小: 175
Epoch 0:   0% 0/351 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:6044: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 0: 352it [09:17,  1.58s/it, loss=0.2331, lr=1.00e-04]
Epoch 0: 

In [23]:
!git fetch origin && git reset --hard origin/main

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 401 bytes | 401.00 KiB/s, done.
From https://github.com/lvyirgil/maimai-chat-ai
   af80070..12043b4  main       -> origin/main
HEAD is now at 12043b4 Add --resume argument support to train_gpu.py
